In [1]:
from pyspark.sql.functions import col, lit, avg
import datetime

In [2]:
user_log_data = sc.textFile("hdfs://localhost:54310/user_log_data/user_log_data.csv").map(lambda x:x.split(","))

In [3]:
# creating dataframe from csv file
# csv filtered to extract columns and values
# in schema index 0 is to collect first row, which is headers
user_log_dataframe = sqlContext.createDataFrame(data=user_log_data.filter(lambda x:x[0]!='user_name'),
                             schema=user_log_data.filter(lambda x:x[0]=='user_name').collect()[0])

In [12]:
# creating new column list to store extracted seconds and username(as primary key) from dataframe 
new_column_list = []
for row in user_log_dataframe.collect():
    # converting string type of datetime to datetime
    date_time = datetime.datetime.strptime(row['working_hours'], "%Y-%m-%d %H:%M:%S")
    converted_seconds = date_time.hour * 3600 + date_time.minute *60 + date_time.second
    new_column_list.append((row['user_name'], converted_seconds))

In [13]:
# creating new column dataframe to join with original dataframe( user_log_dataframe)
new_column = sqlContext.createDataFrame(new_column_list, ('username',"seconds"))

In [14]:
# joining new column to user_log_dataframe using username as primary key
joined_dataframe = (user_log_dataframe
    .join(new_column, col("user_name")==col("username"),"leftouter")
    .drop("username")
   )

In [15]:
# Generating average seconds
average_object = joined_dataframe.agg(avg(col("seconds")))

In [16]:
# unpacking average value from average_object
average_value = average_object.collect()[0][0]

In [25]:
# filtering dataframe with highest average hours users
highest_average_hours = joined_dataframe.filter(joined_dataframe['seconds'] > average_value)

In [24]:
# getting total number of highest average hours users
highest_average_hours.count()

57

In [23]:
# printing top 20 users with highest average hours
highest_average_hours.select("user_name").show()

+--------------------+
|           user_name|
+--------------------+
|ashutoshrit64@gma...|
|giridhardandikwar...|
|salinabodale73@gm...|
|indrajeetgajbhiye...|
|khairnarswapna99@...|
|aman.raj251@gmail...|
|Krushnanikam26@gm...|
|magadum.iranna@gm...|
|puruissimple@gmai...|
|prajktasraut16@gm...|
|samruddhichitnis0...|
|surajwarbhuvan192...|
| grmule018@gmail.com|
|  shelkeva@gmail.com|
|akshaypatwari24@g...|
|rohitsarate1027@g...|
|sharlawar77@gmail...|
|kaleshwetaanil@gm...|
|rahilstar11@gmail...|
|antonyalexcm@gmai...|
+--------------------+
only showing top 20 rows

